# Duplicate fusion

## Imports and settings

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2 as cv
import PIL
import matplotlib.pyplot as plt
import shutil as sh
import time
    
from os import path
from glob import glob

In [ ]:
def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i+1)
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [ ]:
PATH_IN_IMAGES = './in_images/'
PATH_IN_LABELS = './in_labels/'

PATH_OUT_IMAGES = './out_images/'
PATH_OUT_LABELS = './out_labels/'
PATH_OUT_MERGED = './out_merged/'

PATH_DF = './duplicates.xlsx'

In [ ]:
df = pd.read_excel(PATH_DF, header=None)
df

## Single sample check

In [ ]:
# get path of first duplicate pair in source 
f1 = path.join(PATH_IN_LABELS, df[0][19])
f2 = path.join(PATH_IN_LABELS, df[1][19])
print('file 1:', f1)
print('file 2:', f2)

# read as gray image
mask_in1 = cv.imread(f1, cv.IMREAD_GRAYSCALE) 
mask_in2 = cv.imread(f2, cv.IMREAD_GRAYSCALE) 

# merge masks
mask_out = np.logical_or(mask_in1, mask_in2).astype(np.uint8)

# plot results
visualize(mask_1=mask_in1, mask_2=mask_in2, mask_fusion=mask_out)

## Bulk processing

In [ ]:
for index, row in df.iterrows(): 
    print('files:', row[0], row[1])
    
    # get paths of duplicate pair
    f1 = path.join(PATH_IN_LABELS, row[0])
    f2 = path.join(PATH_IN_LABELS, row[1])

    # read as gray image
    mask_in1 = cv.imread(f1, cv.IMREAD_GRAYSCALE) 
    mask_in2 = cv.imread(f2, cv.IMREAD_GRAYSCALE) 

    # merge masks
    mask_out = np.logical_or(mask_in1, mask_in2).astype(np.uint8)

    # plot result
    visualize(mask_1=mask_in1, mask_2=mask_in2, mask_fusion=mask_out)
    
    # save result
    mask_out = PIL.Image.fromarray(mask_out)
    mask_out.save(path.join(PATH_OUT_MERGED, path.basename(f1)))

In [ ]:
# copying original images, removing right side duplicates
sh.copytree(PATH_IN_IMAGES, PATH_OUT_IMAGES, symlinks=False)
time.sleep(10) # wait for copying to finish, DIRTY! (but worky)
[os.remove(path.join(PATH_OUT_IMAGES, row[1])) for _, row in df.iterrows()]

In [ ]:
# copying original labels, removing right side duplicates, writing left side merged
sh.copytree(PATH_IN_LABELS, PATH_OUT_LABELS, symlinks=False)
time.sleep(10) # wait for copying to finish, DIRTY! (but worky)
[os.remove(path.join(PATH_OUT_LABELS, row[1])) for _, row in df.iterrows()]
[sh.copy(path.join(PATH_OUT_MERGED, row[0]), PATH_OUT_LABELS) for _, row in df.iterrows()]